In [1]:
from numcosmo_py import Nc, Ncm, sky_match
Ncm.cfg_init()

from astropy.table import Table, join, unique
import pandas as pd

import sys
sys.path.insert(0, "/global/homes/c/cinlima/gcrcatalogs-new/gcr-catalogs")
import GCRCatalogs
print('GCRCatalogs =', GCRCatalogs.__version__, '|' ,'GCR =', GCRCatalogs.GCR.__version__)

%load_ext autoreload
%autoreload 2

GCRCatalogs = 1.10.2 | GCR = 0.9.2


## Catalogs

In [2]:
#Cosmo DC2
cosmodc2 = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')

#Cosmo DC2 + RedMaPPer
catalog_dc2_redmapper = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')

print("CosmoDC2 info:")
print(f"Version: {cosmodc2.get_catalog_info('version')}")
print(f"Sky area: {cosmodc2.get_catalog_info('sky_area')}")
print(f"Cosmology: {cosmodc2.get_catalog_info('cosmology')}")

# quantities = catalog_dc2_redmapper.list_all_quantities()
# cluster_quantities = [q for q in quantities if 'member' not in q]
# member_quantities = [q for q in quantities if 'member' in q]
    
# cluster_data = Table(catalog_dc2_redmapper.get_quantities(cluster_quantities)) # cDC2 + RM cluster data
# member_data = Table(catalog_dc2_redmapper.get_quantities(member_quantities)) # cDC2 + RM member data

# # Restrictions on mass and redshift
# min_halo_mass = 1e13 #Msun
# redshift_max = 1.2

# quantities_wanted = ['redshift','halo_mass','halo_id','galaxy_id','ra','dec', 'is_central']
# query = GCRCatalogs.GCRQuery('(halo_mass > ' + str(min_halo_mass) +') & (redshift < ' + str(redshift_max) +') & (is_central < ' + str(redshift_max) +') ')
# truth_data = Table(cosmodc2.get_quantities(quantities_wanted, [query]))

# table_halos = truth_data[truth_data['is_central'] == True]

CosmoDC2 info:
Version: 1.1.4
Sky area: 439.78987
Cosmology: {'H0': 71.0, 'Om0': 0.2648, 'Ob0': 0.0448, 'sigma8': 0.8, 'n_s': 0.963}


In [3]:
# CLUSTER DATA

cluster_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/cluster_data.fits')

cluster_member_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/member_data.fits')
cluster_member_data.rename_column('cluster_id_member', 'cluster_id')


In [4]:
# HALO DATA

truth_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/truth_data.fits')

truthdf = truth_data.to_pandas()

hascentral = truthdf[truthdf['is_central'] ==  True]
halo_central_ids = list(hascentral['halo_id'].unique())
central_position = hascentral[['ra', 'dec', 'redshift', 'halo_id']].rename(columns = {'ra': 'halo_ra', 'dec': 'halo_dec', 'redshift': 'halo_redshift'})
halos_dataframe = pd.merge(truthdf[truthdf['halo_id'].isin(halo_central_ids)], central_position, how='inner', on=['halo_id'])


halo_member_data = Table.from_pandas(halos_dataframe)
halo_data =  unique(halo_member_data['halo_id', 'halo_mass', 'halo_ra', 'halo_dec', 'halo_redshift'])

## Matching

In [5]:
halo_coordinates = {"RA":"halo_ra" , "DEC":"halo_dec" , "z":"halo_redshift"}
halo_ids  = {"ID": "halo_id", "MemberID": "galaxy_id"}

detections_coordinates =  {"RA":"cluster_ra" , "DEC":"cluster_dec" , "z":"cluster_redshift"}
detections_ids  = {"ID": "cluster_id", "MemberID": "id_member"}


halos = sky_match.SkyMatch(
    query_data= halo_data,
    query_coordinates=halo_coordinates,
    match_data=cluster_data,
    match_coordinates=detections_coordinates,
    query_member_data=truth_data,
    match_member_data=cluster_member_data,
    query_ids = halo_ids, 
    match_ids = detections_ids
)

detections = halos.invert_query_match()

In [6]:
%time halos_matched = halos.match_ID()

CPU times: user 14.3 s, sys: 505 ms, total: 14.8 s
Wall time: 14.9 s


In [7]:
%time best = halos_matched.select_best()

CPU times: user 5.71 s, sys: 68.9 ms, total: 5.78 s
Wall time: 5.8 s


In [8]:
%time best_table = halos_matched.to_table_best(best)

CPU times: user 8.29 ms, sys: 58 μs, total: 8.35 ms
Wall time: 7.43 ms


In [9]:
best_table[:3]

Index,ID,RA,DEC,z,Index_matched,ID_matched,RA_matched,DEC_matched,z_matched
int64,int64,float64,float64,float64,int64,int32,float64,float64,float32
6,80442,57.94542121163374,-33.78088245138828,0.09851626146477921,20752,29091,57.94542121163374,-33.78088245138828,0.11098104
14,181293,69.18918616334467,-43.41185729310556,0.6914193960527157,44743,54348,69.23231434019651,-43.40737480896762,0.5958041
28,100094347,56.02076522849643,-34.111500439861594,0.42810309720887485,20348,17063,56.02076522849643,-34.111500439861594,0.42926645
